# Licenca in podobno
Kodo za algoritem sem našel na github-u: https://gist.github.com/lcpz/fc02cbf6f0108259302ee4b7d9924dbe
Spremembe glede na original:
Izpustil sem originalni uporabniški vmesnik in dodal primer klica funkcije za uporabo v JupyterLab-u.

## License

Copyright 2015 GoogleInc. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

original: https://goo.gl/kkF5SY

Check whether a given variable-length code is uniquely decodable.
This is a direct/naive implementation of the Sardinas-Patterson algorithm.
It can be used to check if e.g. a given phoneme inventory yields unambiguous
transcriptions.

## Uporaba
Najprej definiramo vse potrebne funkcije (poženemo spodnji blok, tako da kliknemo nanj in pritisnemo Ctrl+Enter),
nato poženemo funkcijo IsUniquelyDecodable in ji podamo set kod (npr. IsUniquelyDecodable({'0','01','011','0111'}) )

### Pozor: 
Vhodni podatek je set, ki ne omogoča več enakih elementov. Zato funkcija ne bo ugotovila, da kode ni možno enolično dekodirati v trivialnem primeru, ko sta dve kodi popolnoma enaki. 

In [83]:
def LeftQuotientOfWord(ps, w):
    """Yields the suffixes of w after removing any prefix in ps."""
    for p in ps:
        if w.startswith(p):
            yield w[len(p):]
    return

def LeftQuotient(ps, ws):
    """Returns the set of suffixes of any word in ws after removing any prefix
    in ps. This is the quotient set which results from dividing ws on the
    left by ps."""
    qs = set()
    for w in ws:
        for q in LeftQuotientOfWord(ps, w):
            qs.add(q)
    return qs

def IsUniquelyDecodable(cs):
    """Checks if the set of codewords cs is uniquely decodable via the
    Sardinas-Patterson algorithm."""
    NL, i = len(str(cs)) * len(str(max(len(x) for x in cs))), 1 # Levenstein's upper bound for termination
    s = LeftQuotient(cs, cs)
    s.discard('')
    if len(s) == 0:
        print('Uniquely decodable prefix code.')
        return True
    while '' not in s and len(s & cs) == 0:
        t = LeftQuotient(cs, s) | LeftQuotient(s, cs)
        if t == s or i > NL + 1:
            print('Uniquely decodable.')
            return True
        s = t
        i += 1
    if '' in s:
        print('Dangling empty suffix.')
    for x in s & cs:
        print('Dangling suffix: {}'.format(x))
    return False

Primeri testa kode:

In [92]:
IsUniquelyDecodable({'0','010','101','01','1'})

Dangling suffix: 01
Dangling suffix: 0
Dangling suffix: 1


False

In [85]:
IsUniquelyDecodable({'0','10','110','111'})

Uniquely decodable prefix code.


True

In [86]:
IsUniquelyDecodable({'0','01','011','111'})

Uniquely decodable.


True

In [87]:
IsUniquelyDecodable({'b','c','ab','ac','aab','aac'})

Uniquely decodable prefix code.


True

### Sledenje algoritmu
Tu je še enkrat ista koda, tokrat z dodanimi izpisi, za lažje sledenje delovanju kode

Nekaj komentarjev:

    yield: sestavlja generatorski objekt (seznam rezultatov), ki ga na koncu funkcije vrne prazen return. V funkcije enkrat ali večkrat kličeš yield, ki vsakič na seznam (generatorski objekt) doda nov rezultat. Ko se funkcija zaključi vrne generatorski objekt z rezultati. Rezultate lahko pregledamo s for funkcijo.
    
    set: Večina spremenljivk so seti, ki se obnašajo kot množice: elementi niso oštevilčeni, dva enaka elementa ne moreta obstajati znotraj istega seta. Če poskusiš setu dodati element, ki že obstaja, se ne bo zgodilo nič. Operator | predstavlja unijo, operator & predstavlja presek

In [105]:
def LeftQuotientOfWord(ps, w):
    """Yields the suffixes of w after removing any prefix in ps."""
    print('  LQOW(ps={}, w={})'.format(ps,w))
    for p in ps:
        if w.startswith(p):
            print('    yield {}'.format(w[len(p):]))
            yield w[len(p):]
    print('  -----------------------------------------------')
    return

def LeftQuotient(ps, ws):
    """Returns the set of suffixes of any word in ws after removing any prefix
    in ps. This is the quotient set which results from dividing ws on the
    left by ps."""
    print('LQ(ps={}, ws={})'.format(ps,ws))
    qs = set()
    for w in ws:
        for q in LeftQuotientOfWord(ps, w):
            qs.add(q)
    print('LQend (return {})'.format(qs))
    print('****************************************')
    return qs

def IsUniquelyDecodable(cs):
    """Checks if the set of codewords cs is uniquely decodable via the
    Sardinas-Patterson algorithm."""
    NL, i = len(str(cs)) * len(str(max(len(x) for x in cs))), 1 # Levenstein's upper bound for termination
    s = LeftQuotient(cs, cs)
    s.discard('')
    print('End of first search for prefixes. s={} cs={}'.format(s,cs))
    if len(s) == 0:
        print('Uniquely decodable prefix code.')
        return True
    while '' not in s and len(s & cs) == 0:
        t = LeftQuotient(cs, s) | LeftQuotient(s, cs)
        print('s={} cs={} t={}'.format(s,cs,t))
        if t == s or i > NL + 1:
            print('Uniquely decodable.')
            return True
        s = t
        i += 1
    if '' in s:
        print('Dangling empty suffix.')
    for x in s & cs:
        print('Dangling suffix: {}'.format(x))
    return False

In [107]:
IsUniquelyDecodable({'0','010','101'})

LQ(ps={'101', '0', '010'}, ws={'101', '0', '010'})
  LQOW(ps={'101', '0', '010'}, w=101)
    yield 
  -----------------------------------------------
  LQOW(ps={'101', '0', '010'}, w=0)
    yield 
  -----------------------------------------------
  LQOW(ps={'101', '0', '010'}, w=010)
    yield 10
    yield 
  -----------------------------------------------
LQend (return {'', '10'})
****************************************
End of first search for prefixes. s={'10'} cs={'101', '0', '010'}
LQ(ps={'101', '0', '010'}, ws={'10'})
  LQOW(ps={'101', '0', '010'}, w=10)
  -----------------------------------------------
LQend (return set())
****************************************
LQ(ps={'10'}, ws={'101', '0', '010'})
  LQOW(ps={'10'}, w=101)
    yield 1
  -----------------------------------------------
  LQOW(ps={'10'}, w=0)
  -----------------------------------------------
  LQOW(ps={'10'}, w=010)
  -----------------------------------------------
LQend (return {'1'})
***************************

False